### Trie class: Insertion and Searching functions

In [1]:
class TrieNode:
    # Trie node class
    def __init__(self):
        self.children = [None] * 128  # Assuming ASCII characters
        self.wordCount = 0
        self.isEndOfWord = False

    def insert_key(self, key):
        # Initialize the currentNode pointer with the root node
        currentNode = self
        
        # Iterate across the length of the string
        for c in key:
            index = ord(c)
            if currentNode.children[index] is None:
                # If node for current character does not exist then make a new node
                newNode = TrieNode()
                # Keep the reference for the newly created node.
                currentNode.children[index] = newNode
            
            # Now, move the current node pointer to the newly created node.
            currentNode = currentNode.children[index]
        
        # Increment the wordEndCount for the last currentNode pointer. 
        # This implies that there is a string ending at currentNode.
        currentNode.wordCount += 1
        currentNode.isEndOfWord = True

    def is_prefix_exist(self, key):
        # Initialize the currentNode pointer with the root node
        current_node = self
        
        # Iterate across the length of the string
        for c in key:
            # Check if the node exists for the current character in the Trie.
            index = ord(c)
            if current_node.children[index] is None:
                # Given word as a prefix does not exist in Trie
                return False
            
            # Move the currentNode pointer to the already existing node for the current character.
            current_node = current_node.children[index]
        
        # Prefix exists in the Trie
        return True

### Installing 'words' package from 'corpus' library

In [2]:
import nltk
#nltk.download('words')

### Importing the 'words' package (corpus/ dictionary)

In [3]:
from nltk.corpus import words

In [4]:
len(words.words())

236736

In [5]:
root = TrieNode()
for word in words.words():
    root.insert_key(word)

print(root.is_prefix_exist('play'))

True


### Input sentence

In [6]:
input_sentence='During the summer we have the best ueather. I have a black ueather jacket, so nice.'

# TEXT PREPROCESSING (FOR ONE INPUT SENTENCE)

### Convert text to lower case

In [7]:
clean_text = input_sentence.lower()
clean_text

'during the summer we have the best ueather. i have a black ueather jacket, so nice.'

### Removing all characters that are not letters or spaces

In [8]:
import re

# Variable to replace all characters that are not letters or whitespace
regex = re.compile('[^a-z\s]')
# Removes all characters that are not letters or spaces
clean_text=regex.sub('', clean_text)
clean_text

'during the summer we have the best ueather i have a black ueather jacket so nice'

### Word Tokenization: Splitting the sentence into list of words

In [9]:
from nltk.tokenize import word_tokenize
input_words = word_tokenize(clean_text)
input_words

['during',
 'the',
 'summer',
 'we',
 'have',
 'the',
 'best',
 'ueather',
 'i',
 'have',
 'a',
 'black',
 'ueather',
 'jacket',
 'so',
 'nice']

## Finding misspelt words and generating candidate words for them

In [10]:
def generate_candidates(word):
    # Generate candidate words for a misspelled word
    candidates = set()
    
    # Insertion: Add a character at each position
    for i in range(len(word) + 1):
        for c in ascii_lowercase:
            candidates.add(word[:i] + c + word[i:])
    
    # Deletion: Remove a character at each position
    for i in range(len(word)):
        candidates.add(word[:i] + word[i+1:])
    
    # Substitution: Replace each character with another character
    for i in range(len(word)):
        for c in ascii_lowercase:
            candidates.add(word[:i] + c + word[i+1:])
    
    # Transposition: Swap adjacent characters
    for i in range(len(word) - 1):
        candidates.add(word[:i] + word[i+1] + word[i] + word[i+2:])
    
    return candidates

def find_candidates(trie_root, sentence):
    candidates = []
    words = sentence.split()
    
    for word in words:
        if not trie_root.is_prefix_exist(word):
            # Generate candidates for misspelled word
            word_candidates = generate_candidates(word)
            # Check each candidate if it exists in the trie
            for candidate in word_candidates:
                if trie_root.is_prefix_exist(candidate):
                    candidates.append(candidate)
    return candidates

In [11]:
from string import ascii_lowercase
for word in input_words:
    misspelled_candidates = find_candidates(root, word)
    print("Word:", word)
    print("Misspelled word's candidates:", misspelled_candidates)
    print()

Word: during
Misspelled word's candidates: []

Word: the
Misspelled word's candidates: []

Word: summer
Misspelled word's candidates: []

Word: we
Misspelled word's candidates: []

Word: have
Misspelled word's candidates: []

Word: the
Misspelled word's candidates: []

Word: best
Misspelled word's candidates: []

Word: ueather
Misspelled word's candidates: ['teather', 'leather', 'feather', 'weather', 'heather', 'neather', 'yeather']

Word: i
Misspelled word's candidates: []

Word: have
Misspelled word's candidates: []

Word: a
Misspelled word's candidates: []

Word: black
Misspelled word's candidates: []

Word: ueather
Misspelled word's candidates: ['teather', 'leather', 'feather', 'weather', 'heather', 'neather', 'yeather']

Word: jacket
Misspelled word's candidates: []

Word: so
Misspelled word's candidates: []

Word: nice
Misspelled word's candidates: []



## Using edit distance, we find the word to replace from the candidate list

In [15]:
def edit_distance(word1, word2):
    # Calculate the edit distance between two words using dynamic programming
    m, n = len(word1), len(word2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Initialize the dp matrix
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
    
    # Calculate the edit distance
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1]) + 1
    
    return dp[m][n]

def choose_correct_word(original_word, candidates):
    # Choose the correct word based on the context using minimum edit distance
    min_distance = float('inf')
    correct_word = None
    
    for candidate in candidates:
        distance = edit_distance(original_word, candidate)
        if distance < min_distance:
            min_distance = distance
            correct_word = candidate
        print(min_distance,candidate)
    return correct_word

for word in input_words:
    if not root.is_prefix_exist(word):
        # Generate candidates for misspelled word
        word_candidates = find_candidates(root, word)
        # Choose the correct word based on the context
        correct_word = choose_correct_word(word, word_candidates)
        print("Misspelled word:", word)
        print("Correct word:", correct_word)

1 teather
1 leather
1 feather
1 weather
1 heather
1 neather
1 yeather
Misspelled word: ueather
Correct word: teather
1 teather
1 leather
1 feather
1 weather
1 heather
1 neather
1 yeather
Misspelled word: ueather
Correct word: teather


In [16]:
### above approach is not semantically correct

# Adding semantic value

### Approach: Each candidate word is compared upon the semantic simalarity wrt to the given sentence and the highest valued candidate word is used to replace the missplelt words

In [27]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Tokenize text and preprocess
tokenized_corpus = [word_tokenize(word.lower()) for word in words.words()]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4)

# Save word embeddings to file
word2vec_model.wv.save_word2vec_format('word2vec_embeddings.bin', binary=True)


In [35]:
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np

# Load pre-trained Word2Vec embeddings
word_vectors = KeyedVectors.load_word2vec_format('word2vec_embeddings.bin', binary=True)

def compute_sentence_semantic_similarity(sentence, candidate_word):
    # Tokenize the input sentence
    tokens = word_tokenize(sentence.lower())
    
    # Compute semantic similarity with each token in the sentence
    similarities = []
    for token in tokens:
        if token in word_vectors and candidate_word in word_vectors:
            similarity = word_vectors.similarity(token, candidate_word)
            similarities.append(similarity)
    
    # Calculate the average similarity
    if similarities:
        avg_similarity = np.mean(similarities)
    else:
        avg_similarity = 0.0  # If no similarities found
    
    return avg_similarity

'''# Example usage
sentence = "The quick brown fox jumps over the lazy dog"
candidate_word = "cat"
semantic_similarity = compute_sentence_semantic_similarity(sentence, candidate_word)
print(f"Semantic similarity of '{candidate_word}' in the sentence: {semantic_similarity}")
'''

for word in input_words:
    if not root.is_prefix_exist(word):
        # Generate candidates for misspelled word
        word_candidates = find_candidates(root, word)
        # Choose the correct word based on the context
        most_similar=None
        similarity_value=float('-inf')
        for i in word_candidates:
            semantic_similarity = compute_sentence_semantic_similarity(clean_text, i)
            print('Here,',semantic_similarity,i)
            if semantic_similarity>similarity_value:
                similarity_value=semantic_similarity
                most_similar=i

        print("Misspelled word:", word)
        print("Similar word:", most_similar)

Here, -0.0026996215 teather
Here, -0.013643089 leather
Here, -0.036639273 feather
Here, -0.029646605 weather
Here, -0.02142083 heather
Here, 0.0 neather
Here, 0.039927017 yeather
Misspelled word: ueather
Similar word: yeather
Here, -0.0026996215 teather
Here, -0.013643089 leather
Here, -0.036639273 feather
Here, -0.029646605 weather
Here, -0.02142083 heather
Here, 0.0 neather
Here, 0.039927017 yeather
Misspelled word: ueather
Similar word: yeather
